<a href="https://colab.research.google.com/github/irwanmazlin/OCR/blob/main/OCR%20PDF%20to%20CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
!pip install timm
!pip install pypdfium2 pillow
!pip install paddlepaddle paddleocr
!pip install img2table
!pip install tabula-py
!pip install pypdf
!pip install jpype1
!pip install pdfkit


from huggingface_hub import hf_hub_download
from transformers import AutoImageProcessor, TableTransformerForObjectDetection

from transformers import AutoModelForObjectDetection
from transformers import DonutProcessor, VisionEncoderDecoderModel
import pandas as pd
import torch
from PIL import Image
import pypdfium2 as pdfium
import numpy as np
from paddleocr import PaddleOCR
import os
import cv2



from paddleocr import PaddleOCR
import os
import cv2
import re

# import pdfkit
import pandas as pd
import matplotlib.pyplot as plt


import cv2
import matplotlib.pyplot as plt






ocr_engine = PaddleOCR(use_angle_cls=True, lang='en')

donut_processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
donut_model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
donut_model.to(device)

pdf_path = pdfium.PdfDocument("/content/TT_FaultCode.pdf")
num_page = len(pdf_path)





count_cell = 0
count_crop = 0
count = 0

all_dataframes = []
listimage = []
all_data = []
all_dataframe = []
for page_num in range(num_page):
  page = pdf_path[page_num]
  image = page.render(scale=3).to_pil()
  # image = page.render(scale=3).to_pil()

  detection_processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")
  detection_model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection")

  inputs = detection_processor(images=image, return_tensors='pt')
  outputs = detection_model(**inputs)

  target_sizes = torch.tensor([image.size[::-1]])
  results = detection_processor.post_process_object_detection(outputs, threshold=0.8, target_sizes=target_sizes)[0]

  table_boxes = results['boxes'][results['labels'] == 0]

  processor_structure = AutoImageProcessor.from_pretrained("bilguun/table-transformer-structure-recognition")
  model_structure = AutoModelForObjectDetection.from_pretrained('bilguun/table-transformer-structure-recognition')





  for box in table_boxes:
      box = [round(i, 2) for i in box.tolist()]
      x_min_table, y_min_table, x_max_table, y_max_table = box
      crop_img = image.crop(box)

      inputs = processor_structure(images=crop_img, return_tensors="pt")
      outputs = model_structure(**inputs)

      target_sizes1 = torch.tensor([crop_img.size[::-1]])
      structure_results = processor_structure.post_process_object_detection(outputs, threshold=0.8, target_sizes=target_sizes1)[0]


      print(len(structure_results))

      crop_img_np = np.array(crop_img)
      cv2.imwrite(f"whole_.jpg", crop_img_np)

      # fig, ax = plt.subplots()
      # ax.imshow(crop_img)
      crop_img_np = np.array(crop_img)

      height, width, _ = crop_img_np.shape

      table_column_name = f'columns/table{count}'
      os.makedirs(table_column_name, exist_ok = True)

      os.makedirs(f"vertical_cells", exist_ok = True)

      header_row = []
      table_data = []
      column_count = 0
      for cell in structure_results['boxes']:
          xmin, ymin, xmax, ymax = map(int, cell.tolist())
          cell_img = crop_img.crop((xmin, ymin, xmax, ymax))
          cell_img_np = np.array(cell_img)
          # result = ocr_engine.ocr(cell_img_np, cls=True)
          # extracted_text = result[0][0][1][0] if result else ""

          # if ymin < 0.2 * crop_img.height:  # Adjust threshold if needed
          #     header_row.append(extracted_text)
          # else:
          #     table_data.append(extracted_text)

          # xmin, ymin, xmax, ymax = map(int, cell.tolist())
          # cell = [round(i, 2) for i in cell.tolist()]
          # crop_img_pil = crop_img.crop(cell)

          # crop_img_np2 = np.array(crop_img_pil)

          xmin = max(0, xmin)
          ymin = max(0, ymin)
          xmax = min(width, xmax)
          ymax = min(height, ymax)

          cell_height = ymax - ymin
          cell_width = xmax - xmin
          aspect_ratio = cell_height / cell_width

          if aspect_ratio > 1.5:
            os.makedirs(f"column_{column_count}", exist_ok=True)
            crop = crop_img_np[ymin: ymax, xmin: xmax]

            gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


            # Detect horizontal lines
            horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
            horizontal_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

            # Find contours for potential horizontal line regions
            contours, _ = cv2.findContours(horizontal_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)



            line_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 100]
            line_contours.sort(key=lambda cnt: cv2.boundingRect(cnt)[1])
            #accumulate text for the entire column
            column_text = []

            for i in range(len(line_contours)):
              top_line = line_contours[i]
              x, y1, w, h1 = cv2.boundingRect(top_line)
              y1 = max(0, y1 + h1)  # Start below the top line

              if i < len(line_contours) - 1:
                  bottom_line = line_contours[i + 1]
                  _, y2, _, h2 = cv2.boundingRect(bottom_line)
                  y2 = min(crop.shape[0], y2)
              else:  # For the last row
                  y2 = crop.shape[0]  # Bottom line is the image's bottom

              if y2 > y1:
                  cropped_region = crop[y1:y2, x:x + w]
                  # print(f"column_{column_count}")

                  cv2.imwrite(f"column_{column_count}/cropped_region_{i}.jpg", cropped_region)

                  result = ocr_engine.ocr(cropped_region, cls = False)
                  text_in_image = []
                  column_text = []
                  for j, item in enumerate(result):
                    textcombine = []
                    for text_info in item:
                      text = text_info[1][0]
                      # print(text)
                      textcombine.append(text)
                    text_in_image.append(' '.join(textcombine))

                  column_text.extend(text_in_image)

                  while len(table_data) <= column_count:
                      table_data.append([])
                  table_data[column_count].extend(text_in_image)

            cv2.imwrite(f"columns/column_{count}.jpg", crop)
            count = count + 1
            column_count = column_count + 1



          else:

            pass  # Placeholder for now



  print(table_data)

  # Create the dataframe with inner lists as columns
  df = pd.DataFrame(table_data).T

  # Rename the columns
  columns_names = ['J1587 PID SID {FMI}', 'Effect', 'J1939 SPN(S) {FMI}', 'Reason', 'Fault Code {Lamp}']
  df.columns = columns_names[:len(df.columns)]
  if not df.empty:
    all_dataframes.append(df)


if all_dataframes:
  final_df = pd.concat(all_dataframes, ignore_index=True)
  final_df.to_csv(f"all_data.csv")

else:
  print("No dataframes to concatenate.")

# Display the first 5 rows
print(final_df)


[2024/09/12 06:19:08] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mo

3
[2024/09/12 06:19:38] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.08418154716491699
[2024/09/12 06:19:38] ppocr DEBUG: rec_res num  : 2, elapsed : 0.2216169834136963
[2024/09/12 06:19:38] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.014708280563354492
[2024/09/12 06:19:38] ppocr DEBUG: rec_res num  : 2, elapsed : 0.08829593658447266
[2024/09/12 06:19:38] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.015150070190429688
[2024/09/12 06:19:38] ppocr DEBUG: rec_res num  : 2, elapsed : 0.08555102348327637
[2024/09/12 06:19:38] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.015116453170776367
[2024/09/12 06:19:38] ppocr DEBUG: rec_res num  : 2, elapsed : 0.10826611518859863
[2024/09/12 06:19:38] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.013675212860107422
[2024/09/12 06:19:38] ppocr DEBUG: rec_res num  : 2, elapsed : 0.08671808242797852
[2024/09/12 06:19:38] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.014915943145751953
[2024/09/12 06:19:38] ppocr DEBUG: rec_res num  : 2, elapsed : 0.0835616588592529

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[['S254 {12}', 'P190 {2}', 'P102 {3}', 'P102 {4}', 'P102 {0}', 'P091 {3}', 'P091 {4}', 'P029 {3}', 'P029 {4}', 'P100 {3}', 'P100 {4}', 'P100 {1}', 'P110 {3}', 'P110 {4}', 'P110 {0}', 'P091 {1}', 'P091 {O}'], ['Possible no noticeable performance effects, engine dying,or hard starting', 'Fueling to injectors is disabled and the engine can not be started.', 'Engine power derate.', 'Engine power derate.', 'Engine power derate.', 'Severe derate in power output of the engine. Limp home power only.', 'Severe derate in power output of the engine.Limp home power only.', 'Remote accelerator will not operate. Remote accel erator position will be set to zero percent', 'Remote accelerator will not operate. Remote accel erator position will be set to zero percent.', 'None on performance. No engine protection for oil pressure.', 'None on performance. No engine protection for oil pressure.', 'None on performance.', 'Possible white smoke.Fan will stay ON if controllec by ECM.No engine protection for en

...

[['P168 {1}', 'P168 {0}', 'P168 {0}', 'P157 {0}', 'P157 {3}', 'P157 {4}', 'P105 {0}', 'S114 {2}', 'P089 {2}', 'P154 {3}', 'P093 {2}', 'S051 {3}', '$230 {4}', 'P157 {0}', 'P157 {2}', 'P157 {1}', '$039 {3}', '$039 {4}', 'P167 {0}'], ['Engine may stop running or be difficult to start.', 'Possible electrical damage to all electrical compo nents.', 'Engine will only idle', 'None or possible engine noise associated with higher injection pressures (especially at idle or light load). Engine power is reduced.', 'Power and or speed derate', 'Power and or speed derate.', 'Progressive power derate increasing in severity from time of alert.', 'Multi-unit synchronization feature is disabled.', 'Intermediate speed control switch may not operate correctly.', 'None on performance', 'Torque curve setting defauits to the default torque curve.', 'None on performance', 'Engine will only idle', 'None or possible engine noise associated with higher injection pressures (especially at idle or ligh load).Engine

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[['P168 {1}', 'P168 {0}', 'P168 {0}', 'P157 {0}', 'P157 {3}', 'P157 {4}', 'P105 {0}', 'S114 {2}', 'P089 {2}', 'P154 {3}', 'P093 {2}', 'S051 {3}', '$230 {4}', 'P157 {0}', 'P157 {2}', 'P157 {1}', '$039 {3}', '$039 {4}', 'P167 {0}'], ['Engine may stop running or be difficult to start.', 'Possible electrical damage to all electrical compo nents.', 'Engine will only idle', 'None or possible engine noise associated with higher injection pressures (especially at idle or light load). Engine power is reduced.', 'Power and or speed derate', 'Power and or speed derate.', 'Progressive power derate increasing in severity from time of alert.', 'Multi-unit synchronization feature is disabled.', 'Intermediate speed control switch may not operate correctly.', 'None on performance', 'Torque curve setting defauits to the default torque curve.', 'None on performance', 'Engine will only idle', 'None or possible engine noise associated with higher injection pressures (especially at idle or ligh load).Engine

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


3
[2024/09/12 06:21:30] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.02236151695251465
[2024/09/12 06:21:30] ppocr DEBUG: rec_res num  : 2, elapsed : 0.1616513729095459
[2024/09/12 06:21:30] ppocr DEBUG: dt_boxes num : 4, elapsed : 0.15026044845581055
[2024/09/12 06:21:30] ppocr DEBUG: rec_res num  : 4, elapsed : 0.3378779888153076
[2024/09/12 06:21:30] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.03711724281311035
[2024/09/12 06:21:31] ppocr DEBUG: rec_res num  : 2, elapsed : 0.14502358436584473
[2024/09/12 06:21:31] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.027624130249023438
[2024/09/12 06:21:31] ppocr DEBUG: rec_res num  : 2, elapsed : 0.15082836151123047
[2024/09/12 06:21:31] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.04259514808654785
[2024/09/12 06:21:31] ppocr DEBUG: rec_res num  : 2, elapsed : 0.1532890796661377
[2024/09/12 06:21:31] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.03504538536071777
[2024/09/12 06:21:31] ppocr DEBUG: rec_res num  : 2, elapsed : 0.1528165340423584
[2024